# Argumentation framework

In [32]:
import os
from pathlib import Path
import subprocess

def run(input, problem, arg = None):
    exe = Path().joinpath(
        os.getcwd(),
        "..",
        "target",
        "release",
        "ter.exe",
    ).resolve()
    result = subprocess.run(
        [
            exe,
            "-p",
            problem,
            "-f",
            input,
            *([] if arg is None else ["-a", arg]),
        ],
        capture_output=True,
        text=True,
    )
    if result.returncode != 0:
        raise Exception(result.stderr)
    return result.stdout

In [33]:
print(run("../examples/tgf.txt", "SE-CO"))

[1,4,6]



In [34]:
print(run("../examples/tgf.txt", "EE-CO"))

[[1,4,6],[1],[3,1]]



In [35]:
print(run("../examples/tgf.txt", "DC-CO", "1"))

YES



In [36]:
print(run("../examples/tgf.txt", "DC-CO", "3"))

YES



In [37]:
print(run("../examples/tgf.txt", "DS-CO", "1"))

YES



In [38]:
print(run("../examples/tgf.txt", "DS-CO", "3"))

NO

